In [1]:
import pandas as pd
import numpy as np

# file
import handset_model_current as handset_model

Using TensorFlow backend.


In [2]:
df = handset_model.load_data()  # set ALL=True in file

In [3]:
y = df["TARGET_S_TO_S_APPLE"]
df = df.drop(['Unnamed: 0', 'ID', 'MPP_NET_DISCOUNT_OTHER_FEE', "TARGET_S_TO_S_APPLE"], axis=1)

y.shape, df.shape

((583291,), (583291, 344))

In [4]:
# all cat cols
CATEGORICAL_COLS = ["CU_GENDER", "CU_EMAIL_IND", "CU_SMS_IND",
                         "CU_MAP_SEGMENT", "CLM_LIVSFASE_SEGMENT", "CU_FYLKE",
                         "CU_HOUSE_TYPE", "HH_MOB_COVERAGE_IN", "HH_MOB_COVERAGE_OUT",
                         "CU_ADSL_OK_RESULT", "CU_VDSL_OK_RESULT", "CU_LINE_STATUS",
                         "CU_U_MAIN_BRAND", "CU_U_MAIN_SUBS_TYPE", "CU_U_MAIN_DEV_MODELNAME",
                         "CU_U_MAIN_DEV_OS_TYPE", "CU_U_MAIN_DEV_PRODUCERNAME", "CU_U_MAIN_DEV_CATEGORY",
                         "CU_U_MAIN_DEV_TYPE", "CU_U_MAIN_DEV_HD_VOICE",
                         "CU_U_MAIN_DEV_TOUCH_SCREEN", "CU_U_MAIN_DEV_LTE", "MPP_BRAND",
                         "MPP_SUBS_START_REASON", "MPP_CHANGETYPE", "MPP_PRODUCT_NAME",
                         "MPP_CHURN_SEGMENT_ADJ", "MPP_DEVICE_MODELNAME", "MPP_DEVICE_OS_TYPE",
                         "MPP_DEVICE_PRODUCERNAME", "MPP_DEVICE_CATEGORY", "MPP_DEVICE_TYPE",
                         "MPP_DEVICE_HD_VOICE", "MPP_DEVICE_TOUCH_SCREEN", "MPP_DEVICE_LTE",
                         "MPP_BINDING_PRODUCT_DESC", "MPP_BINDING_TYPE", "MPP_DEALER_NAME",
                         "MPP_SALES_CH", "MPP_DEALER_CHAIN", "MPP_IN_PORT_SP_NAME"]

In [5]:
for var in CATEGORICAL_COLS:
    df.loc[df.loc[:,var].isnull(), var] = "UNKNOWN"

In [6]:
# TODO: reduce dimension of mpp device modelname and mpp dealer name

df.loc[df.MPP_DEVICE_OS_TYPE.str.contains("MICROSOFT"),"MPP_DEVICE_OS_TYPE"] = "MICROSOFT"
df.loc[df.MPP_DEVICE_OS_TYPE.str.contains("SYMBIAN"),"MPP_DEVICE_OS_TYPE"] = "SYMBIAN"
df.loc[df.MPP_DEVICE_OS_TYPE.str.contains("BLACKBERRY"),"MPP_DEVICE_OS_TYPE"] = "BLACKBERRY"
df.loc[df.MPP_DEVICE_OS_TYPE.isin(['LINUX MAEMO', 'ASHA', 'PROPRIETARY OS', 'BADA']),"MPP_DEVICE_OS_TYPE"] = "OTHER"
df.loc[:, "COUNT"] = df.groupby(["CU_U_MAIN_DEV_PRODUCERNAME"]).CU_U_MAIN_DEV_PRODUCERNAME.transform('count')
df.loc[df.COUNT<200,"CU_U_MAIN_DEV_PRODUCERNAME"] = "OTHER"
df.loc[:, "COUNT"] = df.groupby("MPP_DEVICE_PRODUCERNAME").MPP_DEVICE_PRODUCERNAME.transform('count')
df.loc[df.COUNT<200,"MPP_DEVICE_PRODUCERNAME"] = "OTHER"
df.loc[df.CU_U_MAIN_DEV_PRODUCERNAME=="OTHER","CU_U_MAIN_DEV_MODELNAME"] = "OTHER"
df.loc[df.CU_U_MAIN_DEV_MODELNAME=="IPHONE 5 (A1429)", "CU_U_MAIN_DEV_MODELNAME"]="IPHONE 5"
df.loc[df.CU_U_MAIN_DEV_MODELNAME=="IPHONE 5S (A1457)", "CU_U_MAIN_DEV_MODELNAME"]="IPHONE 5S"
df.loc[df.CU_U_MAIN_DEV_MODELNAME=="IPHONE 6S (A1688 / A1691 / A1700 / A1633)", "CU_U_MAIN_DEV_MODELNAME"]="IPHONE 6S"
df.loc[df.CU_U_MAIN_DEV_MODELNAME=="IPHONE 5C (A1532 / A1456)", "CU_U_MAIN_DEV_MODELNAME"]="IPHONE 5C"
df.loc[df.CU_U_MAIN_DEV_MODELNAME=="GT-I9506 (GALAXY S4 LTE-A)", "CU_U_MAIN_DEV_MODELNAME"]="GT-I9505 (GALAXY S4 LTE)"
df.loc[df.CU_U_MAIN_DEV_MODELNAME=="SM-G920F (SM-G920I GALAXY S6)", "CU_U_MAIN_DEV_MODELNAME"]="SM-G920F ( GALAXY S6)"
df.loc[df.CU_U_MAIN_DEV_MODELNAME=="SM-G925F (SM-G925I GALAXY S6 EDGE)", "CU_U_MAIN_DEV_MODELNAME"]="SM-G925F( GALAXY S6 EDGE)"
df.loc[:, "COUNT"] = df.groupby(["CU_U_MAIN_DEV_PRODUCERNAME", "CU_U_MAIN_DEV_MODELNAME"]).CU_U_MAIN_DEV_MODELNAME.transform('count')
brands = df.CU_U_MAIN_DEV_PRODUCERNAME.unique()
for brand in brands:
    df.loc[(df.COUNT<1000)&(df.CU_U_MAIN_DEV_PRODUCERNAME==brand), "CU_U_MAIN_DEV_MODELNAME"] = brand + "_other"

In [7]:
len(df.MPP_DEALER_NAME.unique())

3214

In [8]:
df.loc[:, 'MPP_DEALER_NAME'] = df.MPP_DEALER_NAME.str.lower()
df.loc[:, 'MPP_DEALER_NAME'].head()

0                        neras
1                       eureka
2    telenorbutikken fornebu s
3      telenorbutikken cc vest
4                        neras
Name: MPP_DEALER_NAME, dtype: object

In [9]:
# example function to see if lots of different values start with the same word (company)
for k in df.MPP_DEALER_NAME.unique():
    if 'elprice' in k.lower():
        print(k)

elprice haugesund
elprice nettbutikk
elprice sartor
elprice sandnes
elprice knarvik as
elprice fana
elprice førde
elprice vinterbro
elprice sandefjord
elprice avd. ålesund
elprice gjøvik
elprice åsane
elprice strømmen
elprice fredrikstad
elprice as porsgrunn
elprice as lørenskog
elprice sentrum
elprice moss
elprice bryne
elprice trondheim
elprice sørlandsparken/kristiansand
elprice ski
elprice sandvika
elprice tønsberg
elprice avd. tiller
elprice molde


In [10]:
df.loc[df.MPP_DEALER_NAME.str.contains("tele"),"MPP_DEALER_NAME"] = "TELENOR"
df.loc[df.MPP_DEALER_NAME.str.contains("elkjøp"),"MPP_DEALER_NAME"] = "ELKJØP"
df.loc[df.MPP_DEALER_NAME.str.contains("spaceworld"),"MPP_DEALER_NAME"] = "SPACEWORLD"
df.loc[df.MPP_DEALER_NAME.str.contains("expert"),"MPP_DEALER_NAME"] = "EXPERT"
df.loc[df.MPP_DEALER_NAME.str.contains("mobildata"),"MPP_DEALER_NAME"] = "MOBILDATA"
df.loc[df.MPP_DEALER_NAME.str.contains("lefdal"),"MPP_DEALER_NAME"] = "LEFDAL"
df.loc[df.MPP_DEALER_NAME.str.contains("nordialog"),"MPP_DEALER_NAME"] = "NORDIALOG"
df.loc[df.MPP_DEALER_NAME.str.contains("coop"),"MPP_DEALER_NAME"] = "COOP"
df.loc[df.MPP_DEALER_NAME.str.contains("elektrosenter"),"MPP_DEALER_NAME"] = "ELEKTROSENTER"
df.loc[df.MPP_DEALER_NAME.str.contains("elprice"),"MPP_DEALER_NAME"] = "ELPRICE"

df.loc[:, "COUNT"] = df.groupby(["MPP_DEALER_NAME"]).MPP_DEALER_NAME.transform('count')
df.loc[df.COUNT<200,"MPP_DEALER_NAME"] = "OTHER"
# mpp dealer name now has 85 unique values (instead of ~3000!)

In [11]:
len(df.MPP_DEALER_NAME.unique())

85

In [12]:
len(df.MPP_DEVICE_MODELNAME.unique())

806

In [13]:
df.loc[df.MPP_DEVICE_MODELNAME=="IPHONE 5 (A1429)", "MPP_DEVICE_MODELNAME"]="IPHONE 5"
df.loc[df.MPP_DEVICE_MODELNAME=="IPHONE 5S (A1457)", "MPP_DEVICE_MODELNAME"]="IPHONE 5S"
df.loc[df.MPP_DEVICE_MODELNAME=="IPHONE 6S (A1688 / A1691 / A1700 / A1633)", "MPP_DEVICE_MODELNAME"]="IPHONE 6S"
df.loc[df.MPP_DEVICE_MODELNAME=="IPHONE 5C (A1532 / A1456)", "MPP_DEVICE_MODELNAME"]="IPHONE 5C"
df.loc[df.MPP_DEVICE_MODELNAME=="GT-I9506 (GALAXY S4 LTE-A)", "MPP_DEVICE_MODELNAME"]="GT-I9505 (GALAXY S4 LTE)"
df.loc[df.MPP_DEVICE_MODELNAME=="SM-G920F (SM-G920I GALAXY S6)", "MPP_DEVICE_MODELNAME"]="SM-G920F ( GALAXY S6)"
df.loc[df.MPP_DEVICE_MODELNAME=="SM-G925F (SM-G925I GALAXY S6 EDGE)", "MPP_DEVICE_MODELNAME"]="SM-G925F( GALAXY S6 EDGE)"
df.loc[:, "COUNT"] = df.groupby(["MPP_DEVICE_PRODUCERNAME", "MPP_DEVICE_MODELNAME"]).CU_U_MAIN_DEV_MODELNAME.transform('count')
brands = df.MPP_DEVICE_PRODUCERNAME.unique()
for brand in brands:
    df.loc[(df.COUNT<1000)&(df.MPP_DEVICE_PRODUCERNAME==brand), "MPP_DEVICE_MODELNAME"] = brand + "_other"
    
# mpp device modelname now has 78 unique values (instead of ~800)

In [14]:
len(df.MPP_DEVICE_MODELNAME.unique())

78